In [1]:
import pandas as pd
import re
from langdetect import detect 

In [2]:
df1 = pd.read_csv('./datasets/data_sci.csv')
df1.head()

,id,title,text,subreddit
0,a7clej,"Elon Musk Stomps About The Tesla Factory ""Rage...",NaN,sci
1,a7ceq2,Scientists discover the most distant solar sys...,NaN,sci
2,a7cdlr,"Feminists Think Sexist Men Are Sexier than ""Wo...",NaN,sci
3,a7ccqv,The average primary care doctor needs to work ...,NaN,sci
4,a7ca1s,Fossil from the Big Bang Discovered with W. M....,NaN,sci


In [3]:
df1.shape

(10000, 4)

In [4]:
df1['id'].duplicated().sum()

0

In [5]:
df2 = pd.read_csv('./datasets/data_psy.csv')
df2.head()

,id,title,text,subreddit
0,a7cisf,KNOW YOUR WORTH,NaN,psy
1,a7bzh3,The Real Roots of American Rage - The untold s...,NaN,psy
2,a7b4tk,Is it Social Anxiety?,NaN,psy
3,a7b2hc,Trying to find something good in a bad situati...,NaN,psy
4,a7b0c5,Children of problem drinkers are more likely t...,NaN,psy


In [6]:
df2.shape

(9991, 4)

In [7]:
df2['id'].duplicated().sum()

0

In [8]:
df = pd.concat([df1, df2])
df.reset_index(drop=True, inplace=True)
df.head()

,id,title,text,subreddit
0,a7clej,"Elon Musk Stomps About The Tesla Factory ""Rage...",NaN,sci
1,a7ceq2,Scientists discover the most distant solar sys...,NaN,sci
2,a7cdlr,"Feminists Think Sexist Men Are Sexier than ""Wo...",NaN,sci
3,a7ccqv,The average primary care doctor needs to work ...,NaN,sci
4,a7ca1s,Fossil from the Big Bang Discovered with W. M....,NaN,sci


In [9]:
df.tail()

,id,title,text,subreddit
19986,720vgx,Our Ability to Recognize Letters Could Be Hard...,NaN,psy
19987,720o8i,Building a World Without Alzheimer's,[deleted],psy
19988,720o5a,i hear voices in my head (schizophrenia)|| La ...,NaN,psy
19989,720fvv,The Fallout of Not Facing the Toxic Behaviors ...,NaN,psy
19990,72071z,What's that woman thinking?,NaN,psy


In [10]:
df.shape

(19991, 4)

In [11]:
df['subreddit'].value_counts()

sci    10000
psy     9991
Name: subreddit, dtype: int64

In [12]:
df['text'].fillna('', inplace=True)
df.head()

,id,title,text,subreddit
0,a7clej,"Elon Musk Stomps About The Tesla Factory ""Rage...",,sci
1,a7ceq2,Scientists discover the most distant solar sys...,,sci
2,a7cdlr,"Feminists Think Sexist Men Are Sexier than ""Wo...",,sci
3,a7ccqv,The average primary care doctor needs to work ...,,sci
4,a7ca1s,Fossil from the Big Bang Discovered with W. M....,,sci


In [13]:
mask = df['text'].str.contains('\[deleted')
df.loc[mask, 'text'] = ''
    
mask = df['title'].str.contains('\[deleted')
df.drop(df.loc[mask].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [14]:
df.shape

(19986, 4)

In [15]:
df['content'] = df['title'] + df['text']
df.drop(['title', 'text'], axis=1, inplace=True)
df.head()

,id,subreddit,content
0,a7clej,sci,"Elon Musk Stomps About The Tesla Factory ""Rage..."
1,a7ceq2,sci,Scientists discover the most distant solar sys...
2,a7cdlr,sci,"Feminists Think Sexist Men Are Sexier than ""Wo..."
3,a7ccqv,sci,The average primary care doctor needs to work ...
4,a7ca1s,sci,Fossil from the Big Bang Discovered with W. M....


In [16]:
def preprocess_text(text):
         
    urls_removed = re.sub('http\S+', ' ', text)
                          
    letters_only = re.sub('[^a-zA-Z]', ' ', urls_removed)
    
    words = letters_only.lower().split()
    
    return(' '.join(words))

In [17]:
df['content'] = df['content'].map(lambda x: preprocess_text(x))

In [18]:
mask = df['content'] == ''
df.drop(df.loc[mask].index, inplace=True)

In [19]:
mask = df['content'].map(lambda x: True if detect(x) != 'en' else False)
df.drop(df.loc[mask].index, inplace=True)

In [20]:
df['subreddit'] = df['subreddit'].map(lambda x: 1 if x == 'sci' else 0)

In [21]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df.head()

,id,subreddit,content
0,9uqrxt,1,cigar shaped interstellar object may have been...
1,a3bo24,0,research based framework for the popular advic...
2,a30vp0,1,your fat and gut hormone interact to tell your...
3,76xc3u,0,opposition to same sex marriage is often drive...
4,9mnhla,1,in a huge world first study australian scienti...


In [22]:
df.shape

(18481, 3)

In [23]:
df.to_csv('./datasets/data_all.csv', index=False)